In [41]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({})



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({})



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 135)

In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({})



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
        [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 135)

In [6]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({})



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 135)

In [8]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({})



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 135)

In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({})



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 135)

In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({})



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            ])
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

SyntaxError: invalid syntax (<ipython-input-3-da9b8367097c>, line 42)

In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({})



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            ])
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    )

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 136)

In [6]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            ])
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    )

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

SyntaxError: invalid syntax (<ipython-input-6-deb8182049f0>, line 42)

In [7]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            ])
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: unexpected indent (<ipython-input-7-adc0a67ac92a>, line 69)

In [8]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            ])
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: unexpected indent (<ipython-input-8-b9cda7b4b084>, line 69)

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            ])
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    
#    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: unexpected indent (<ipython-input-1-b9cda7b4b084>, line 69)

In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            ])
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    
#    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: unexpected indent (<ipython-input-2-b9cda7b4b084>, line 69)

In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            ])
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    
    html.Hr(),
        dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: unexpected indent (<ipython-input-3-e6ee9254639c>, line 70)

In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            ])
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    
    html.Hr(),
        dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: unexpected indent (<ipython-input-4-ccba0eff2b3e>, line 69)

In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: unexpected indent (<ipython-input-5-ccba0eff2b3e>, line 69)

In [6]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

SyntaxError: invalid syntax (<ipython-input-6-9f62486b158c>, line 73)

In [7]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: expected an indented block (<ipython-input-7-1a157480959e>, line 155)

In [8]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
    #return [
    #    dcc.Graph(            
    #        figure = pie
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

AttributeError: 'AnimalShelter' object has no attribute 'read'

In [9]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
    #return [
    #    dcc.Graph(            
    #        figure = pie
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

OperationFailure: Authentication failed.

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
    #return [
    #    dcc.Graph(            
    #        figure = pie
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

OperationFailure: Authentication failed.

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
    #return [
    #    dcc.Graph(            
    #        figure = pie
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

OperationFailure: Authentication failed.

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
    #return [
    #    dcc.Graph(            
    #        figure = pie
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

OperationFailure: Authentication failed.

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
    #return [
    #    dcc.Graph(            
    #        figure = pie
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

OperationFailure: Authentication failed.

In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
    #return [
    #    dcc.Graph(            
    #        figure = pie
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

AttributeError: 'AnimalShelter' object has no attribute 'readDoc'

In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
    #return [
    #    dcc.Graph(            
    #        figure = pie
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

AttributeError: 'AnimalShelter' object has no attribute 'readDoc'

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
    #return [
    #    dcc.Graph(            
    #        figure = pie
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

NameError: name 'base64' is not defined

In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
   return [
       dcc.Graph(            
            figure = pie
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: unexpected indent (<ipython-input-2-c97fdb262dc5>, line 154)

In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
   return [
       dcc.Graph(            
            figure = pie
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

FileNotFoundError: [Errno 2] No such file or directory: 'my-image.png'

In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
   return [
       dcc.Graph(            
            figure = pie
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
   return [
       dcc.Graph(            
            figure = pie
       )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
                     
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
   return [
       dcc.Graph(            
            figure = pie
       )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(className = 'row',
             style = {'display' : 'flex'},
                 children = [
                     html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Water Rescue'),
                     html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Wilderness Rescue')
                     html.Button(id = 'submit-button-three', n_clicks = 0, children = 'Individual Tracking')
                     html.Button(id = 'submit-button-four', n_clicks = 0, children = 'Reset')
                 ]),
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('submit-button-one', 'n_clicks'), Input('submit-button-two', 'n_clicks'), Input('submit-button-three', 'n_clicks'), Input('submit-button-four', 'n_clicks')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
   return [
       dcc.Graph(            
            figure = pie
       )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

SyntaxError: invalid syntax (<ipython-input-2-91215298c53e>, line 62)

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)



# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(
        html.P([
            html.Label("Filters"),
            dcc.RadioItems(
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
        ])
    ),

#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('submit-button-one', 'n_clicks'), Input('submit-button-two', 'n_clicks'), Input('submit-button-three', 'n_clicks'), Input('submit-button-four', 'n_clicks')])
def update_dashboard(bt1, bt2, bt3, bt4):
### FIX ME Add code to filter interactive data table with MongoDB queries

#        if (int(bt1) == 0 and int(bt2) == 0 and int(bt3) == 0 and int(bt4) == 0):
#            df = pd.DataFrame.from_records(shelter.readDoc({}))
        
#        elif (int(bt2) > int(bt1) and int(bt2) > int(bt3) and int(bt2) > int(bt4)):
            

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
   return [
       dcc.Graph(            
            figure = pie
       )    
    ]
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

SyntaxError: invalid syntax (<ipython-input-1-d7fb738618fd>, line 114)

In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)



# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(
        html.P([
            html.Label("Filters"),
            dcc.RadioItems(
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
        ])
    ),

#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('submit-button-one', 'n_clicks'), Input('submit-button-two', 'n_clicks'), Input('submit-button-three', 'n_clicks'), Input('submit-button-four', 'n_clicks')])
def update_dashboard(bt1, bt2, bt3, bt4):
### FIX ME Add code to filter interactive data table with MongoDB queries

#        if (int(bt1) == 0 and int(bt2) == 0 and int(bt3) == 0 and int(bt4) == 0):
#            df = pd.DataFrame.from_records(shelter.readDoc({}))
        
#        elif (int(bt2) > int(bt1) and int(bt2) > int(bt3) and int(bt2) > int(bt4)):
            

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
#   gf = pd.DataFrame.from_dict(viewData)
#   gff = gf.loc[gf['breed']]
#   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
#   return [
#       dcc.Graph(            
#            figure = pie
#       )    
#    ]
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)



# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(
        html.P([
            html.Label("Filters"),
            dcc.RadioItems(
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
        ])
    ),

#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('submit-button-one', 'n_clicks'), Input('submit-button-two', 'n_clicks'), Input('submit-button-three', 'n_clicks'), Input('submit-button-four', 'n_clicks')])
def update_dashboard(bt1, bt2, bt3, bt4):
### FIX ME Add code to filter interactive data table with MongoDB queries

#        if (int(bt1) == 0 and int(bt2) == 0 and int(bt3) == 0 and int(bt4) == 0):
#            df = pd.DataFrame.from_records(shelter.readDoc({}))
        
#        elif (int(bt2) > int(bt1) and int(bt2) > int(bt3) and int(bt2) > int(bt4)):
            

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
#   gf = pd.DataFrame.from_dict(viewData)
#   gff = gf.loc[gf['breed']]
#   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
#   return [
#       dcc.Graph(            
#            figure = pie
#       )    
#    ]
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

IndentationError: expected an indented block (<ipython-input-3-cedf84d3fefc>, line 174)

In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)



# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(
        html.P([
            html.Label("Filters"),
            dcc.RadioItems(
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
        ])
    ),

#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('submit-button-one', 'n_clicks'), Input('submit-button-two', 'n_clicks'), Input('submit-button-three', 'n_clicks'), Input('submit-button-four', 'n_clicks')])
def update_dashboard(bt1, bt2, bt3, bt4):
### FIX ME Add code to filter interactive data table with MongoDB queries

#        if (int(bt1) == 0 and int(bt2) == 0 and int(bt3) == 0 and int(bt4) == 0):
#            df = pd.DataFrame.from_records(shelter.readDoc({}))
        
#        elif (int(bt2) > int(bt1) and int(bt2) > int(bt3) and int(bt2) > int(bt4)):
            

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#@app.callback(
#    Output('graph-id', "children"),
#    [Input('datatable-id', "derived_viewport_data")])
#def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
#   gf = pd.DataFrame.from_dict(viewData)
#   gff = gf.loc[gf['breed']]
#   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
#   return [
#       dcc.Graph(            
#            figure = pie
#       )    
#    ]
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)



# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(
            dcc.RadioItems(
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    
    ),

#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('submit-button-one', 'n_clicks'), Input('submit-button-two', 'n_clicks'), Input('submit-button-three', 'n_clicks'), Input('submit-button-four', 'n_clicks')])
def update_dashboard(bt1, bt2, bt3, bt4):
### FIX ME Add code to filter interactive data table with MongoDB queries

#        if (int(bt1) == 0 and int(bt2) == 0 and int(bt3) == 0 and int(bt4) == 0):
#            df = pd.DataFrame.from_records(shelter.readDoc({}))
        
#        elif (int(bt2) > int(bt1) and int(bt2) > int(bt3) and int(bt2) > int(bt4)):
            

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
   return [
       dcc.Graph(            
            figure = pie
       )    
    ]
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [6]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)



# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div(
            dcc.RadioItems(
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    
    ),

#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('submit-button-one', 'n_clicks'), Input('submit-button-two', 'n_clicks'), Input('submit-button-three', 'n_clicks'), Input('submit-button-four', 'n_clicks')])
def update_dashboard(bt1, bt2, bt3, bt4):
### FIX ME Add code to filter interactive data table with MongoDB queries

#        if (int(bt1) == 0 and int(bt2) == 0 and int(bt3) == 0 and int(bt4) == 0):
#            df = pd.DataFrame.from_records(shelter.readDoc({}))
        
#        elif (int(bt2) > int(bt1) and int(bt2) > int(bt3) and int(bt2) > int(bt4)):
            

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
                               
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
   gf = pd.DataFrame.from_dict(viewData)
   gff = gf.loc[gf['breed']]
   pie = go.Pie(labes = gf['animal_type'], values = gf['breed'])
   
   return [
       dcc.Graph(            
            figure = pie
       )    
    ]
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({})



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#@app.callback(
#    Output('graph-id', "children"),
#    [Input('datatable-id', "derived_viewport_data")])
#def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

SyntaxError: invalid syntax (<ipython-input-1-e192ff59cbb7>, line 42)

In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records')
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#@app.callback(
#    Output('graph-id', "children"),
#    [Input('datatable-id', "derived_viewport_data")])
#def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

SyntaxError: invalid syntax (<ipython-input-2-b3ad7a626e95>, line 42)

In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#@app.callback(
#    Output('graph-id', "children"),
#    [Input('datatable-id', "derived_viewport_data")])
#def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

SyntaxError: invalid syntax (<ipython-input-3-5e598877ffaf>, line 78)

In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#@app.callback(
#    Output('graph-id', "children"),
#    [Input('datatable-id', "derived_viewport_data")])
#def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

AttributeError: 'AnimalShelter' object has no attribute 'read'

In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#@app.callback(
#    Output('graph-id', "children"),
#    [Input('datatable-id', "derived_viewport_data")])
#def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

AttributeError: 'AnimalShelter' object has no attribute 'read'

In [6]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#@app.callback(
#    Output('graph-id', "children"),
#    [Input('datatable-id', "derived_viewport_data")])
#def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

NameError: name 'base64' is not defined

In [7]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#@app.callback(
#    Output('graph-id', "children"),
#    [Input('datatable-id', "derived_viewport_data")])
#def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

NameError: name 'base64' is not defined

In [8]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#@app.callback(
#    Output('graph-id', "children"),
#    [Input('datatable-id', "derived_viewport_data")])
#def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [9]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):

    dff = pd.DataFrame.from_dict(viewData)
    
    new_df = df.value_counts('breed')
    
    names = ['breed']
    
    return [
        dcc.Graph(
            
            figure = px.pie(new_df, values = new_df, names = names)

        )
    ]
        
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [10]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):

    dff = pd.DataFrame.from_dict(viewData)
    
    new_df = df.value_counts('breed')
    
    names = ['breed']
    
    return [
        dcc.Graph(
            
            figure = px.pie(new_df, values = new_df, names = names)

        )
    ]
        
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):

    dff = pd.DataFrame.from_dict(viewData)
    
    new_df = df.value_counts('breed')
    
    names = ['breed']
    
    return [
        dcc.Graph(
            
            figure = px.pie(new_df, values = new_df, names = names)

        )
    ]
        
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    graphValues = [ 500, 1500, 300, 150]
    
    names = ['A', 'B', 'C', 'D']
    
    return [
        dcc.Graph(
            
            figure = px.pie(values = graphValues, names = names)

        )
    ]
        
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    graphValues = [ 500, 1500, 300, 150]
    
    names = ['A', 'B', 'C', 'D']
    
    return [
        dcc.Graph(
            
            figure = px.pie(values = graphValues, names = names)

        )
    ]
        
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    graphValues = pd.DataFrame(list(viewData))
    #[ 500, 1500, 300, 150]
    
    #names = ['A', 'B', 'C', 'D']
    
    return [
        dcc.Graph(
            
            figure = px.pie(graphValues, values = 'animal_id', names = 'breed')

        )
    ]
        
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    graphValues = [ 500, 1500, 300, 150]
    
    names = ['A', 'B', 'C', 'D']
    
    return [
        dcc.Graph(
            
            figure = px.pie(values = graphValues, names = names)

        )
    ]
        
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    graphValues = [ 500, 1500, 300, 150]
    
    names = ['A', 'B', 'C', 'D']
    
    return [
        dcc.Graph(
            
            figure = px.pie(values = graphValues, names = names)

        )
    ]
        
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = '/Desktop/Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    graphValues = [ 500, 1500, 300, 150]
    
    names = ['A', 'B', 'C', 'D']
    
    return [
        dcc.Graph(
            
            figure = px.pie(values = graphValues, names = names)

        )
    ]
        
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = '/Desktop/Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    graphValues = [ 500, 1500, 300, 150]
    
    names = ['A', 'B', 'C', 'D']
    
    return [
        dcc.Graph(
            
            figure = px.pie(values = graphValues, names = names)

        )
    ]
        
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

FileNotFoundError: [Errno 2] No such file or directory: '/Desktop/Grazioso Salvare Logo.png'

In [6]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Desktop/Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    graphValues = [ 500, 1500, 300, 150]
    
    names = ['A', 'B', 'C', 'D']
    
    return [
        dcc.Graph(
            
            figure = px.pie(values = graphValues, names = names)

        )
    ]
        
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

FileNotFoundError: [Errno 2] No such file or directory: 'Desktop/Grazioso Salvare Logo.png'

In [7]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Desktop/Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    graphValues = [ 500, 1500, 300, 150]
    
    names = ['A', 'B', 'C', 'D']
    
    return [
        dcc.Graph(
            
            figure = px.pie(values = graphValues, names = names)

        )
    ]
        
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

FileNotFoundError: [Errno 2] No such file or directory: 'Desktop/Grazioso Salvare Logo.png'

In [8]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    graphValues = [ 500, 1500, 300, 150]
    
    names = ['A', 'B', 'C', 'D']
    
    return [
        dcc.Graph(
            
            figure = px.pie(values = graphValues, names = names)

        )
    ]
        
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app

In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter

from crud1 import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readDoc({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Felix Vargas'))),
    html.Hr(),
    html.Div([
            dcc.RadioItems(
            id = 'filters',
                options = [
                {'label' : 'Mountain', 'value' : 'Mountain'},
                {'label' : 'Water Rescue', 'value' : 'Water Rescue'},
                {'label' : 'Disaster', 'value' : 'Disaster'},
                {'label' : 'Reset', 'value' : 'Reset'}
                ],
            value = 'Reset',
            labelStyle = {'display' : 'inline-block'}
            )
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', "data"),
    [Input('filters', "value")])

def render_charts(value):
    
    mountains = {"animal_type" : "Dog", "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    water = {"animal_type" : "Dog", "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks" : {"$gte" : 26}, "age_upon_outcome_in_weeks" : {"$lte" : 156}}
    disaster = {"animal_type" : "Dog", "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiller"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks" : {"$gte" : 20}, "age_upon_outcome_in_weeks" : {"$lte" : 300}}
    
    if value == 'Mountain':
        
        df = pd.DataFrame.from_records(shelter.readDoc(mountains))
        
    elif value == 'Water Rescue':
    
        df = pd.DataFrame.from_records(shelter.readDoc(water))
        
    elif value == 'Disaster':
    
        df = pd.DataFrame.from_records(shelter.readDoc(disaster))
        
    elif value == 'Reset':
    
        df = pd.DataFrame.from_records(shelter.readDoc({}))
        
    
                                       
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    graphValues = [ 500, 1500, 300, 150]
    
    names = ['A', 'B', 'C', 'D']
    
    return [
        dcc.Graph(
            
            figure = px.pie(values = graphValues, names = names)

        )
    ]
        
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    #Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width' : '1000px', 'height' : '500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and pop up
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1 ,9])
                ])
            ])
        ])
    ]


app